<a href="https://colab.research.google.com/github/MuskanSinhaPi/Intern/blob/main/Copy_of_STHSL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -rf STHSL

In [ ]:
!git clone https://github.com/MuskanSinhaPi/STHSL

Cloning into 'STHSL'...
remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 101 (delta 41), reused 82 (delta 38), pack-reused 9
Receiving objects: 100% (101/101), 7.07 MiB | 12.72 MiB/s, done.
Resolving deltas: 100% (41/41), done.


In [ ]:
%cd STHSL

/content/STHSL


##Data Preprocessing for SF Crime Dataset

In [ ]:
import pandas as pd
import numpy as np
import pickle
from datetime import datetime
from sklearn.preprocessing import LabelEncoder

# Load the dataset (adjust the path to your dataset)
sf_crime = pd.read_csv('/content/STHSL/Datasets/SF_crime/SFCrime.csv')

sf_crime = sf_crime[['Category', 'Descript', 'DayOfWeek', 'Date', 'Time', 'PdDistrict','Resolution', 'Address', 'X','Y']]

#Data Cleaning
# Drop rows with any missing values
sf_crime.dropna(inplace=True)
# Convert 'Date' and 'Time' columns to datetime because we can only use .dt accessor with datetimelike values
sf_crime['datetime'] = pd.to_datetime(sf_crime['Date'] + ' ' + sf_crime['Time'])

# Extract relevant features
sf_crime['year'] = sf_crime['datetime'].dt.year
sf_crime['hour'] = sf_crime['datetime'].dt.hour
sf_crime['month'] = sf_crime['datetime'].dt.month
sf_crime['day_of_week'] = sf_crime['datetime'].dt.dayofweek
sf_crime['day'] = pd.to_datetime(sf_crime['datetime']).dt.day
sf_crime['minute'] = pd.to_datetime(sf_crime['datetime']).dt.minute
# Filter data for years before 2018 for training and use 2018 for testing
train_df = sf_crime[sf_crime['year'] < 2018]
test_df = sf_crime[sf_crime['year'] == 2018]

label_encoder = LabelEncoder()

# Encode 'Category' (target variable)
train_df['Category'] = label_encoder.fit_transform(train_df['Category'])
test_df['Category'] = label_encoder.transform(test_df['Category'])

features = ['PdDistrict','hour','day','minute','day_of_week', 'month', 'X', 'Y']
X_train = train_df[features]
y_train = train_df['Category']
X_test = test_df[features]
y_test = test_df['Category']

# One-hot encode the 'PdDistrict' feature
X_train = pd.get_dummies(X_train, columns=['PdDistrict'])
X_test = pd.get_dummies(X_test, columns=['PdDistrict'])

# Ensure the columns match between train and test sets
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

# Combine train and test data to ensure all unique regions are captured
all_data = pd.concat([train_df, test_df])
unique_regions = all_data[['X', 'Y']].drop_duplicates().reset_index(drop=True)
region_map = {tuple(region): idx for idx, region in unique_regions.iterrows()}

# Function to create a region ID
def get_region_id(row):
    # Handle cases where (X, Y) pair is not found in region_map
    return region_map.get((row['X'], row['Y']), -1)  # Assign -1 for unknown regions

train_df['region'] = train_df.apply(get_region_id, axis=1)
test_df['region'] = test_df.apply(get_region_id, axis=1)

# Pivot table to reshape data
train_pivot = train_df.pivot_table(index='region', columns='datetime', values=features, aggfunc='mean')
test_pivot = test_df.pivot_table(index='region', columns='datetime', values=features, aggfunc='mean')

# Fill NaNs with 0
train_pivot = train_pivot.fillna(0)
test_pivot = test_pivot.fillna(0)

# Convert to numpy arrays
train_array = train_pivot.to_numpy()
test_array = test_pivot.to_numpy()

# Assume args.row and args.col are predefined based on the unique regions
args.row = len(unique_regions)
args.col = 1  # Adjust based on your dataset's structure
args.offNum = len(features)

# Reshape the arrays
train_array = train_array.reshape((args.row, args.col, -1, args.offNum))
test_array = test_array.reshape((args.row, args.col, -1, args.offNum))

# Save the arrays as .pkl files
with open('/content/STHSL/Datasets/SF_crime/trn.pkl', 'wb') as f:
    pickle.dump(train_array, f)
with open('/content/STHSL/Datasets/SF_crime/val.pkl', 'wb') as f:
    pickle.dump(test_array, f)
with open('/content/STHSL/Datasets/SF_crime/tst.pkl', 'wb') as f:
    pickle.dump(test_array, f)

# Reshape the arrays as required by the model (example: [region, days, features])
# This step will depend on how your model expects the data to be shaped

ValueError: Cannot set a DataFrame with multiple columns to the single column region

In [ ]:
!python train.py --data SF

self.predir Datasets/SF_crime/
Traceback (most recent call last):
  File "/content/STHSL/train.py", line 51, in <module>
    main()
  File "/content/STHSL/train.py", line 11, in main
    engine = trainer(device)
  File "/content/STHSL/engine.py", line 12, in __init__
    self.handler = DataHandler()
  File "/content/STHSL/DataHandler.py", line 23, in __init__
    args.row, args.col, _, args.offNum = trnT.shape
ValueError: not enough values to unpack (expected 4, got 2)
